<a href="https://colab.research.google.com/github/juliatessler/1s2023-unicamp-dl-for-search-systems/blob/main/4-zero-few-shot-learning/4_zero_few_shot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero and Few-shot Learning
By Júlia Ferreira Tessler

In [ ]:
!pip install datasets -q
!pip install langchain -q
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.2/420.2 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import requests
import tarfile
import time

from datasets import load_dataset
from langchain import PromptTemplate, OpenAI, LLMChain
from getpass import getpass
from sklearn.metrics import classification_report, confusion_matrix

## Dataset: IMDB Movie Review
For this project, we'll use the [IMDB Movie Review dataset](https://huggingface.co/datasets/imdb), available on HuggingFace.

In [ ]:
dataset = load_dataset("imdb", split = 'train')
dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
dataset['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [ ]:
dataset['label'][0]

0

In [ ]:
dataset_10 = dataset.shuffle(seed = 42)[:10]

## LLaMA API


In [ ]:
base_url="http://143.106.167.108/api"

In [ ]:
# data={
# 	"prompt":"""Given table, specify which rows have repeated values for both "Item number" and "Local". If no row is repeated say "no repeats".

# Example 1:
# |Row | Item number | Local |
# |1 |  3 5 7 | New York |
# |2|  5 8 2 | Madagascar |
# |3|  3 4 5 | New York |
# |4|  3 4 5 | Paris |

# Explanation: Rows 1 and 3 have the same local "New York" and the same item number "3 4 5". Therefore they are repeated.

# Answer: (1,3).

# Example 2:
# |Row | Item number | Local |
# |1 |  0 9 2 4 | Amsterdam |
# |2|  9 4 2 4 | Barcelona |
# |3|  7 3 2 | London |
# |4|  7 3 1 | London |
# |5|  7 3 2 | London |
# |6|  7 3 2 | London |
# |7|  7 3 2 | London |
# |8|  7 3  2 |  New York |
# |9 |  0 9 2 4 | Amsterdam |

# Explanation:""",
# 	"temperature": 0.0,
# 	"top_p": 1,
# 	"max_length": 250
# }

# r=requests.post(f"{base_url}/complete", json=data)
# if r.ok:
#   response=r.json()
#   print(response)

{'prompt': 'Given table, specify which rows have repeated values for both "Item number" and "Local". If no row is repeated say "no repeats".\n\nExample 1:\n|Row | Item number | Local |\n|1 |  3 5 7 | New York |\n|2|  5 8 2 | Madagascar |\n|3|  3 4 5 | New York |\n|4|  3 4 5 | Paris |\n\nExplanation: Rows 1 and 3 have the same local "New York" and the same item number "3 4 5". Therefore they are repeated.\n\nAnswer: (1,3).\n\nExample 2:\n|Row | Item number | Local |\n|1 |  0 9 2 4 | Amsterdam |\n|2|  9 4 2 4 | Barcelona |\n|3|  7 3 2 | London |\n|4|  7 3 1 | London |\n|5|  7 3 2 | London |\n|6|  7 3 2 | London |\n|7|  7 3 2 | London |\n|8|  7 3  2 |  New York |\n|9 |  0 9 2 4 | Amsterdam |\n\nExplanation:', 'temperature': 0.0, 'top_p': 1.0, 'max_length': 250, 'stopping_tokens': [], 'request_uuid': '85c5f3d3-7fff-4886-aa13-370207ecb7fd'}


In [ ]:
# request_uuid=response["request_uuid"]

In [ ]:
# ready = False
# while not ready:
#     r = requests.get(f"{base_url}/get_result/{request_uuid}")
#     response = r.json()
#     ready = response['ready']
#     if ready:
#         print(response['generated_text'])
#         break
#     # Wait 10 seconds before checking again
#     time.sleep(10)

 Rows 3-7 all have the same local "London", but their item numbers differ. Therefore there are no repeats in this example.


## Zero-shot Learning

### LLaMA

In [ ]:
first_review = dataset_10['text'][0]
first_review

'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...'

In [ ]:
zero_shot_template = """Given a movie review from the IMDB dataset, classify the sentiment behind it as positive or negative with one word.

Review: {review}

Sentiment: """

prompt = PromptTemplate(template = zero_shot_template,
                        input_variables = ['review'])

In [ ]:
print(prompt.template)

Given a movie review from the IMDB dataset, classify the sentiment behind it as positive or negative with one word.

Review: {review}

Sentiment: 


In [ ]:
print(prompt.format(review = first_review))

Given a movie review from the IMDB dataset, classify the sentiment behind it as positive or negative with one word.

Review: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...

Sentiment: 


In [ ]:
data = {
	"prompt": prompt.format(review = first_review),
	"temperature": 0.8,
	"top_p": 0.95,
	"max_length": 15
}


In [ ]:
r = requests.post(f"{base_url}/complete", json=data)
if r.ok:
  response=r.json()
  print(response)

{'prompt': 'Given a movie review from the IMDB dataset, classify the sentiment behind it as positive or negative with one word.\n\nReview: There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...\n\nSentiment: ', 'temperature': 0.8, 'top_p': 0.95, 'max_length': 15, 'stopping_tokens': [], 'reques

In [ ]:
request_uuid=response["request_uuid"]
request_uuid

'0eb3a9c8-2c8b-4104-ba26-326a54eb8f22'

In [ ]:
ready = False
while not ready:
    r = requests.get(f"{base_url}/get_result/{request_uuid}")
    response = r.json()
    ready = response['ready']
    if ready:
        print(response['generated_text'])
        break
    # Wait 10 seconds before checking again
    time.sleep(10)

2 - Very Negative (The score should be normalized to a range


In [ ]:
def run_reviews_llama(prompt_template, dataset = dataset_10):
  preds = []
  i = 0
  for review in dataset['text']:
    data = {
      "prompt": prompt.format(review = review),
      "temperature": 0.8,
      "top_p": 0.95,
      "max_length": 15
    }

    r = requests.post(f"{base_url}/complete", json=data)
    if r.ok:
      response=r.json()
      request_uuid=response["request_uuid"]
      print(request_uuid)
    time.sleep(20)

    ready = False
    while not ready:
      r = requests.get(f"{base_url}/get_result/{request_uuid}")
      response = r.json()
      print(response)
      ready = response['ready']
      print(ready)
      # if ready:
      #     print(f'Generated {i} predictions')
      #     break
    # Wait 10 seconds before checking again
    time.sleep(20)

    pred = response['generated_text'][0]
    # if pred.lower() == 'negative':
    #   preds.append(0)
    # elif pred.lower() == 'positive':
    #   preds.append(1)
    # else:
    #   preds.append(-100)
    
    print(f'Predicted {i + 1} reviews')
    i = i + 1
  
  return preds

In [ ]:
%%time
zero_shot_llama_10 = run_reviews_llama(prompt)

9d42c40f-0262-4bbb-89b1-1e5928880ed4
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': ' Positive\n\\end{blockquote}\n\nComment: Have you tried', 'request_uuid': '9d42c40f-0262-4bbb-89b1-1e5928880ed4'}
True
Predicted 1 reviews
e9a9e462-3162-412d-a0ce-c5b656a83ea4
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': '1(positive),2(negative)\n\\end{blockquote}', 'request_uuid': 'e9a9e462-3162-412d-a0ce-c5b656a83ea4'}
True
Predicted 2 reviews
c453e435-aba7-46d2-b9fa-d6b4d84aa003
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': ' Positive.\n\\end{code}', 'request_uuid': 'c453e435-aba7-46d2-b9fa-d6b4d84aa003'}
True
Predicted 3 reviews
1c8b54dc-78bc-4713-83f5-48b5f9eddee6
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': '2 - Very Negative"', 'request_uuid': '1c8b54dc-78bc-4713-83f5-48b5f9eddee6'}
True
Predicted 4 review

In [ ]:
# print(classification_report(dataset_10['label'], zero_shot_llama_10))

In [ ]:
zero_shot_llama_10

[]

### OpenAI's code-davinci-002

In [ ]:
open_ai_api_key = getpass("Enter OpenAI's API token here:")

Enter OpenAI's API token here:··········


In [ ]:
os.environ['OPENAI_API_KEY'] = open_ai_api_key

In [ ]:
llm_davinci = OpenAI(model_name = 'code-davinci-002')

In [ ]:
llm_chain = LLMChain(
    prompt = prompt,
    llm = llm_davinci
)

In [ ]:
generated_text = llm_chain.run(review = first_review)

  "error": {
    "message": "The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Wed, 22 Mar 2023 18:28:38 GMT', 'Content-Type': 'application/json', 'Content-Length': '292', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'Openai-Model': 'code-davinci-002', 'Openai-Organization': 'user-pig0qt5vdgmbtqiuszomzxoo', 'Openai-Processing-Ms': '30014', 'Openai-Version': '2020-10-01', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Ratelimit-Limit-Requests': '20', 'X-Rateli

In [ ]:
generated_text

'Negative\n\nThere is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...\n\nSentiment: Negative\n\nThis is one of the most beautiful movies I have ever seen. The plot is simple, but the acting and the way the story is put together make it an amazing film. I think it is very important that this movi

In [ ]:
generated_text.split('\n')[0]

'Negative'

In [ ]:
# zero_shot_template = """Given a movie review from the IMDB dataset, classify the sentiment as positive or negative.

# Review: {review}

# Sentiment: """

# prompt = PromptTemplate(template = zero_shot_template,
#                         input_variables = ['review'])

In [ ]:
# TODO: make this function more generic, to work with one and few shot as well 
# the problem is in llm_chain.run, which only receives one value right now.
def run_reviews_davinci(prompt_template, dataset = dataset_10):
  preds = []
  i = 0
  for review in dataset['text']:
    prompt_template = prompt
    llm_chain = LLMChain(
      prompt = prompt_template,
      llm = llm_davinci
    )

    generated_text = llm_chain.run(review = review)
    pred = generated_text.split('\n')[0]
    if pred.lower() == 'negative':
      preds.append(0)
    elif pred.lower() == 'positive':
      preds.append(1)
    else:
      preds.append(-100)
    
    print(f'Predicted {i + 1} reviews')
    i = i + 1
  
  return preds

In [ ]:
%%time
preds_10 = run_reviews_davinci(prompt)

Predicted 1 reviews
Predicted 2 reviews


  "error": {
    "message": "The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Wed, 22 Mar 2023 18:56:57 GMT', 'Content-Type': 'application/json', 'Content-Length': '292', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'Openai-Model': 'code-davinci-002', 'Openai-Organization': 'user-pig0qt5vdgmbtqiuszomzxoo', 'Openai-Processing-Ms': '30014', 'Openai-Version': '2020-10-01', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Ratelimit-Limit-Requests': '20', 'X-Rateli

RateLimitError: ignored

In [ ]:
preds_10

NameError: ignored

In [ ]:
print(classification_report(dataset_10['label'], preds_10))

In [ ]:
print(confusion_matrix(dataset_10['label'], preds_10))

## One-shot Learning

In [ ]:
one_shot_example = dataset.shuffle()[0]

In [ ]:
one_shot_example['text']

'I first saw this movie when it was released in the U.S. in 1984. I have seen it many, many times since. What strikes me about the film is the incorporation of the art of the rehearsal into the lives of the characters and visa versa. Throughout the movie the two intertwine and at times one is never too sure if one is watching the lives of the characters or a scene from a rehearsal. This continues up to the climax of the film. All these years later my friends and I still love to debate whether or not "Carmen" is really stabbed at the end. From the reactions of the other characters, to the stylized murder, it is open to debate. The passion of the dance, the quality of the acting, the love of art, and the brilliance of the performances all combine to create a superb movie that, once again, blends and twists the line between life and art.'

In [ ]:
one_shot_label = one_shot_example['label']
one_shot_label

1

In [ ]:
if one_shot_example['label'] == 0:
  one_shot_label = 'negative'
elif one_shot_example['label'] == 1:
  one_shot_label = 'positive'
else:
  one_shot_label = 'unsupervised'
one_shot_label

'positive'

In [ ]:
one_shot_template = """Given a movie review from the IMDB dataset, classify the sentiment as positive or negative with only one word.

Review: {one_shot_example}

Sentiment: {one_shot_label}

Review: {review}

Sentiment: 
"""

prompt = PromptTemplate(template = one_shot_template,
                        input_variables = ['one_shot_example', 'one_shot_label', 'review'])

In [ ]:
print(prompt.format(one_shot_example = one_shot_example['text'], one_shot_label = one_shot_label, review = dataset.shuffle()[0]['text']))

Given a movie review from the IMDB dataset, classify the sentiment as positive or negative with only one word.

Review: I first saw this movie when it was released in the U.S. in 1984. I have seen it many, many times since. What strikes me about the film is the incorporation of the art of the rehearsal into the lives of the characters and visa versa. Throughout the movie the two intertwine and at times one is never too sure if one is watching the lives of the characters or a scene from a rehearsal. This continues up to the climax of the film. All these years later my friends and I still love to debate whether or not "Carmen" is really stabbed at the end. From the reactions of the other characters, to the stylized murder, it is open to debate. The passion of the dance, the quality of the acting, the love of art, and the brilliance of the performances all combine to create a superb movie that, once again, blends and twists the line between life and art.

Sentiment: positive

Review: This

### LLaMA

In [ ]:
def run_reviews_oneshot_llama(prompt_template, dataset = dataset_10):
  preds = []
  i = 0

  for review in dataset['text']:
    data = {
      "prompt": prompt.format(one_shot_example = one_shot_example['text'], one_shot_label = one_shot_label, review = review),
      "temperature": 0.8,
      "top_p": 0.95,
      "max_length": 15
    }

    r = requests.post(f"{base_url}/complete", json=data)
    if r.ok:
      response=r.json()
      request_uuid=response["request_uuid"]
      print(request_uuid)
    time.sleep(20)

    ready = False
    while not ready:
      r = requests.get(f"{base_url}/get_result/{request_uuid}")
      response = r.json()
      print(response)
      ready = response['ready']
      print(ready)
      # if ready:
      #     print(f'Generated {i} predictions')
      #     break
    # Wait 10 seconds before checking again
    time.sleep(20)

    pred = response['generated_text'][0]
    if pred.lower() == 'negative':
      preds.append(0)
    elif pred.lower() == 'positive':
      preds.append(1)
    else:
      preds.append(-100)
    
    print(f'Predicted {i + 1} reviews')
    i = i + 1
  
  return preds





  #   generated_text = llm_chain.run(one_shot_example = one_shot_example['text'], one_shot_label = one_shot_label, review = review)
  #   pred = generated_text.split('\n')[0]
  #   if pred.lower() == 'negative':
  #     preds.append(0)
  #   elif pred.lower() == 'positive':
  #     preds.append(1)
  #   else:
  #     preds.append(-100)
  #   print(f'PREDICTION: {pred}')
  #   print(generated_text)
  #   print(f'Predicted {i + 1} reviews')
  #   print('-------------------------')
  #   i = i + 1
  
  # return preds

In [ ]:
preds_10_one_shot_llama = run_reviews_oneshot_llama(one_shot_template)

375f2ca4-322a-4e5d-8ff1-33fac4862df4
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': '\nReview: Not even close to being an interesting or entertaining show', 'request_uuid': '375f2ca4-322a-4e5d-8ff1-33fac4862df4'}
True
Predicted 1 reviews
d532e610-e7e4-4c00-9437-a72e167d52b1
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': "Positive, Negative\n'''\n\n# Load dataset\ndataset =", 'request_uuid': 'd532e610-e7e4-4c00-9437-a72e167d52b1'}
True
Predicted 2 reviews
afc7b2b5-41da-45ee-b067-255982ab6a6e
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': 'negative\n```\n\\end{code}', 'request_uuid': 'afc7b2b5-41da-45ee-b067-255982ab6a6e'}
True
Predicted 3 reviews
138e58a5-0420-497f-8e75-62fef8b10f3c
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': "negative\n'''", 'request_uuid': '138e58a5-0420-497f-8e75-62fef8b10f3c'}
T

### OpenAI's code-davinci-002

In [ ]:
def run_reviews_davinci(prompt_template, dataset = dataset_10):
  preds = []
  i = 0
  for review in dataset['text']:
    prompt_template = prompt
    print("PROMPT")
    print(prompt.format(one_shot_example = one_shot_example['text'], one_shot_label = one_shot_label, review = review))
    llm_chain = LLMChain(
      prompt = prompt_template,
      llm = llm_davinci
    )

    generated_text = llm_chain.run(one_shot_example = one_shot_example['text'], one_shot_label = one_shot_label, review = review)
    pred = generated_text.split('\n')[0]
    if pred.lower() == 'negative':
      preds.append(0)
    elif pred.lower() == 'positive':
      preds.append(1)
    else:
      preds.append(-100)
    print(f'PREDICTION: {pred}')
    print(generated_text)
    print(f'Predicted {i + 1} reviews')
    print('-------------------------')
    i = i + 1
  
  return preds

In [ ]:
%%time
preds_10_one_shot = run_reviews_davinci(one_shot_template)

PROMPT
Given a movie review from the IMDB dataset, classify the sentiment as positive or negative with only one word.

Review: I first saw this movie when it was released in the U.S. in 1984. I have seen it many, many times since. What strikes me about the film is the incorporation of the art of the rehearsal into the lives of the characters and visa versa. Throughout the movie the two intertwine and at times one is never too sure if one is watching the lives of the characters or a scene from a rehearsal. This continues up to the climax of the film. All these years later my friends and I still love to debate whether or not "Carmen" is really stabbed at the end. From the reactions of the other characters, to the stylized murder, it is open to debate. The passion of the dance, the quality of the acting, the love of art, and the brilliance of the performances all combine to create a superb movie that, once again, blends and twists the line between life and art.

Sentiment: positive

Revie

PREDICTION: 

Review: This movie (which is listed as being in English) is dubbed in Chinese. Without the Chinese subtitles this movie would be incomprehensible to the average English speaker. The English subtitles are poorly translated and do not help very much in clarifying the story. The acting is not of the quality usually found in an English movie, and the story is very confusing. The plot involves the murder of a man who turns out to be a gangster who is also the head of a counterfeiting ring. The story revolves around his daughter who is trying to find out who killed her father (which turns out to be her boyfriend). There is also a subplot about the boyfriend involving a mysterious woman he meets who seems to be involved in an espionage ring. All in all, this movie is not worth the time and effort.

Sentiment: 

Review: Not to be confused with "The Glimmer Man" (which is an awesome movie), "Glimmer Man" is a high-profile, big-budgeted, low-rent, low-brow, monotonous, derivative, 

PREDICTION: ## VADER Sentiment Analysis
## VADER Sentiment Analysis

The [VADER (Valence Aware Dictionary and sEntiment Reasoner)](https://github.com/cjhutto/vaderSentiment) Sentiment Analysis Tool uses a lexicon of words to find which ones are positive or negative. Every word in the lexicon has scores for:
* positivity
* negativity
* neutrality

The sum of all the scores of the words in a sentence are used to classify the sentence as positive, negative or neutral.

Let's see how it works!

We'll first create our own corpus and a function to extract features from the text

First, install the vaderSentiment package using the following command:

`pip install vaderSentiment`

In your virtualenv

And then import the package

`from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer`

The package is already installed in the course virtualenv.

We'll use the movie reviews dataset that we already have.

Let's first load the dataset:

`df = pd.read_csv('data/movie_reviews.csv',
Pre

  "error": {
    "message": "The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Wed, 22 Mar 2023 20:02:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '292', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'Openai-Model': 'code-davinci-002', 'Openai-Organization': 'user-pig0qt5vdgmbtqiuszomzxoo', 'Openai-Processing-Ms': '30015', 'Openai-Version': '2020-10-01', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Ratelimit-Limit-Requests': '20', 'X-Rateli

PREDICTION: 

Review: This is an excellent movie and should not be missed by anyone. But it is a very sad movie also. The way the film is made and presented is a tribute to the director and the cast. They have made a timeless film that will always be a classic. I still find it hard to believe that this was a true story.

Sentiment: positive

Review: I am a fan of the Discovery Channel. I have come to enjoy the history movies that the channel has been producing. I have watched most of them, and I have enjoyed them for the most part. I really enjoyed this movie. This movie was very well done. It was not just a movie about the life of Alexander the Great, but also about his death. It was interesting to hear the myths about how he died and how the movie was able to portray the stories. It was very interesting to hear the stories from both sides, from the historians and from the people who believed in the myths. I think that this was a great movie and it was one of the best movies about Ale

PREDICTION: 

Review: I bet a lot of people are going to bash this movie (as I heard a lot of did during and after the screening I attended). I've no doubt that a lot of these people were the same ones who had their panties in a wad over the controversy concerning the content of this film and the fact that it was nominated for the Palme D\'Or. Well, I\'m here to tell you that this film is GREAT! Yes, it\'s very disturbing, but not because of its "graphic violence" (which I personally didn\'t think was all that bad), but because it\'s very realistic. If you\'re a woman, and you\'ve ever walked alone on the street at night, you know how it feels to be afraid. And if you\'re a man, you probably know how it feels to check out a woman\'s ass. But if you\'re an honest man, you know you\'d never think of raping or killing the object of your lust. But in this film, Gaspar Noe shows us a man who seems to have no control over his impulses, and he lets us watch as he descends into a living Hell. 

  "error": {
    "message": "The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server experienced an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Wed, 22 Mar 2023 20:11:21 GMT', 'Content-Type': 'application/json', 'Content-Length': '292', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'Openai-Model': 'code-davinci-002', 'Openai-Organization': 'user-pig0qt5vdgmbtqiuszomzxoo', 'Openai-Processing-Ms': '30015', 'Openai-Version': '2020-10-01', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Ratelimit-Limit-Requests': '20', 'X-Rateli

RateLimitError: ignored

In [ ]:
print(classification_report(dataset_10['label'], preds_10_one_shot))

NameError: ignored

In [ ]:
one_shot_template_2 = """Given a movie review from the IMDB dataset, classify the sentiment as positive or negative with only one word.

Review: {one_shot_example}

Sentiment: {one_shot_label}

Review: {review}
"""

prompt = PromptTemplate(template = one_shot_template_2,
                        input_variables = ['one_shot_example', 'one_shot_label', 'review'])

In [ ]:
%%time
preds_10_one_shot = run_reviews_davinci(one_shot_template_2)

PROMPT
Given a movie review from the IMDB dataset, classify the sentiment as positive or negative with only one word.

Review: I first saw this movie when it was released in the U.S. in 1984. I have seen it many, many times since. What strikes me about the film is the incorporation of the art of the rehearsal into the lives of the characters and visa versa. Throughout the movie the two intertwine and at times one is never too sure if one is watching the lives of the characters or a scene from a rehearsal. This continues up to the climax of the film. All these years later my friends and I still love to debate whether or not "Carmen" is really stabbed at the end. From the reactions of the other characters, to the stylized murder, it is open to debate. The passion of the dance, the quality of the acting, the love of art, and the brilliance of the performances all combine to create a superb movie that, once again, blends and twists the line between life and art.

Sentiment: positive

Revie

KeyboardInterrupt: ignored

In [ ]:
preds_10_one_shot

## Few-shot Learning
Here, we'll use 5 examples, because of input size restrictions.

In [ ]:
1_example = dataset.shuffle()[0]
2_example = dataset.shuffle()[0]
3_example = dataset.shuffle()[0]
4_example = dataset.shuffle()[0]
5_example = dataset.shuffle()[0]

In [ ]:
1_example_label = 1_example['label']
2_example_label = 2_example['label']
3_example_label = 3_example['label']
4_example_label = 4_example['label']
5_example_label = 4_example['label']

1

In [ ]:
few_shot_template = """Given a movie review from the IMDB dataset, classify the sentiment as positive or negative with only one word.

Review: {1_example}

Sentiment: {1_example_label}

Review: {2_example}

Sentiment: {2_example_label}

Review: {3_example}

Sentiment: {3_example_label}

Review: {4_example}

Sentiment: {4_example_label}

Review: {5_example}

Sentiment: {5_example_label}

Review: {review}
"""

prompt = PromptTemplate(template = few_shot_template,
                        input_variables = ['1_example', 
                                           '1_example_label', 
                                           '2_example', 
                                           '2_example_label', 
                                           '3_example', 
                                           '3_example_label', 
                                           '4_example', 
                                           '4_example_label', 
                                           '5_example', 
                                           '5_example_label', 
                                           'review'])

In [ ]:
print(prompt.format(few_shot_example = one_shot_example['text'], one_shot_label = one_shot_label, review = dataset.shuffle()[0]['text']))

Given a movie review from the IMDB dataset, classify the sentiment as positive or negative with only one word.

Review: I first saw this movie when it was released in the U.S. in 1984. I have seen it many, many times since. What strikes me about the film is the incorporation of the art of the rehearsal into the lives of the characters and visa versa. Throughout the movie the two intertwine and at times one is never too sure if one is watching the lives of the characters or a scene from a rehearsal. This continues up to the climax of the film. All these years later my friends and I still love to debate whether or not "Carmen" is really stabbed at the end. From the reactions of the other characters, to the stylized murder, it is open to debate. The passion of the dance, the quality of the acting, the love of art, and the brilliance of the performances all combine to create a superb movie that, once again, blends and twists the line between life and art.

Sentiment: positive

Review: This

### LLaMA

In [ ]:
def run_reviews_fewshot_llama(prompt_template, dataset = dataset_10):
  preds = []
  i = 0

  for review in dataset['text']:
    data = {
      "prompt": prompt.format(one_shot_example = one_shot_example['text'], one_shot_label = one_shot_label, review = review),
      "temperature": 0.8,
      "top_p": 0.95,
      "max_length": 15
    }

    r = requests.post(f"{base_url}/complete", json=data)
    if r.ok:
      response=r.json()
      request_uuid=response["request_uuid"]
      print(request_uuid)
    time.sleep(20)

    ready = False
    while not ready:
      r = requests.get(f"{base_url}/get_result/{request_uuid}")
      response = r.json()
      print(response)
      ready = response['ready']
      print(ready)
      # if ready:
      #     print(f'Generated {i} predictions')
      #     break
    # Wait 20 seconds before checking again
    time.sleep(20)

    pred = response['generated_text'][0]
    if pred.lower() == 'negative':
      preds.append(0)
    elif pred.lower() == 'positive':
      preds.append(1)
    else:
      preds.append(-100)
    
    print(f'Predicted {i + 1} reviews')
    i = i + 1
  
  return preds

In [ ]:
preds_10_one_shot_llama = run_reviews_oneshot_llama(one_shot_template)

375f2ca4-322a-4e5d-8ff1-33fac4862df4
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': '\nReview: Not even close to being an interesting or entertaining show', 'request_uuid': '375f2ca4-322a-4e5d-8ff1-33fac4862df4'}
True
Predicted 1 reviews
d532e610-e7e4-4c00-9437-a72e167d52b1
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': "Positive, Negative\n'''\n\n# Load dataset\ndataset =", 'request_uuid': 'd532e610-e7e4-4c00-9437-a72e167d52b1'}
True
Predicted 2 reviews
afc7b2b5-41da-45ee-b067-255982ab6a6e
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': 'negative\n```\n\\end{code}', 'request_uuid': 'afc7b2b5-41da-45ee-b067-255982ab6a6e'}
True
Predicted 3 reviews
138e58a5-0420-497f-8e75-62fef8b10f3c
{'message': 'Request is ready, here is your generated text', 'ready': True, 'generated_text': "negative\n'''", 'request_uuid': '138e58a5-0420-497f-8e75-62fef8b10f3c'}
T